For this session, we've modified notebooks from several sources:

- [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*
*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

- [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/)

## The IPython/Jupyter notebook 

The notebook interface is designed to be an interactive computing environment, where you can mix code, data, documentation and outputs. This a very good way to document your computational experiments.

A notebook is structured into different _cells_. Each cell contains either (executable) code, an output, or documention (text, images, links ...).

The most important shortcut for you is: <kbd>Shift</kbd> + <kbd>Return</kbd> 
This will execute the code in a cell.

There are many nice tutorials online. If you want to have a closer look at the notebook interface, we would recommend:

- [Tips and Tricks](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/)
- [A longer tutorial](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.Qb=Px60)

A quick test: just enter some simple calculations into the cell below...

In [ ]:
1 + 2

# Linear Regression

## An Overview

||continuous|categorical|
|---|---|---|
|**supervised**|**regression**|classification|
|**unsupervised**|dimension reduction|clustering|


Regression is the task of predicting a continuous (e.g. real-valued) target y from a set of observations x. It is one of the protypical examples of supervised learning. Some simple cases of regression tasks:

- predict temperature
- predict prices
- predict signal value at a single voxel in an MR scan.

Linear regression models are a good starting point for regression tasks.
Such models are popular because they can be fit very quickly, and are very interpretable.

<img src="./img/linear_fit.png" align="middle"/>


You are probably familiar with the simplest form of a linear regression model (i.e., fitting a straight line to data) but such models can be extended to model more complicated data behavior. The linear regression model can be used as long as long as we have a linear function of the _coefficients_, so we can actually fit more complex relationships between x and y. This is done using **basis functions**. These are functions $\phi(x)$ that transform the input variable x and linear regression is done on the transformed $\phi(x)$. Today we will look the simplest case: polynomial kernels, where $\phi_n(x) = x^n$.

The more complex your model is, the greater is the risk of overfitting the data. As can be nicely illustrated by using polynomial models of increasing order. The actual data is shown as blue points, the actual function underlying the data is shown in blue and the estimated polynomial model in green.

![](./img/polynomial-fit-complexity.png)

As you see from the plots above, there seems to be a sweet spot between too simple and too complex models. But how to find the right complexity? Well, here is one way:

You can look at the error on the training and on the test data, to see at which point overfitting occurs. If you use simple models, your training and test error will typically be high. Both will decrease with increasing model compexity. But: at a certain model complexity, the test set error starts to increase again. This is where you would start to overfit your training data. The plots show the model complexity against mean squared error (mse). "Good" models have low training error and a similar test error. (The goals are: "decrease the training error and decrease the gap between training and test error.") In this particular example, polynomial models of orders 6, 7, or 8 would seem like a good choice...
![](./img/polynomial-fit-overfitting.png)

So, when you use more complex models, you should also use regularization to reduce overfitting. Regularizing typically forces your parameters to be small and your results to be smoother and less variable. We will explore this in a bit more detail later...
![](./img/polynomial-fit-regularizer.png)

But you should always keep another option in mind: The more data you collect, the better your fit will be. Collecting more data also regularizes your model fit and reduces the risk of overfitting.
![](./img/polynomial-fit-data-regularization.png)

In this section we will start with a quick intuitive walk-through of the mathematics behind this well-known problem, before seeing how before moving on to see how linear models can be generalized to account for more complicated patterns in data.

So, why are we learning linear regression?
- widely used
- runs fast
- easy to use (not a lot of tuning required)
- highly interpretable
- basis for many other methods


## Estimating ("Learning") Model Coefficients

Generally speaking, coefficients are estimated using the **least squares criterion**, which means we are find the line (mathematically) which minimizes the **sum of squared residuals** (or "sum of squared errors"):

<img src="./img/08_estimating_coefficients.png" style="width: 600px;" align="left">

What elements are present in the diagram?
- The black dots are the <span style="color:black;">**observed values**</span> of x and y. 
- The blue line is our <span style="color:blue;">**least squares line**</span>.
- The red lines are the <span style="color:red;">**residuals**</span>, which are the distances between the observed values and the least squares line.

How do the model coefficients relate to the least squares line?
- $w_0$ is the **intercept** (the value of $y$ when $x$=0)
- $w_1$ is the **slope** (the change in $y$ divided by change in $x$)
- with $y = w_1 x + w_0$

Here is a graphical depiction of those calculations:

<img src="./img/08_slope_intercept.png" style="width: 400px;" align="left">

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

## Simple Linear Regression

We will start with the most familiar linear regression, a straight-line fit to data.
A straight-line fit is a model of the form $y = w_1 x + w_0$

where $w_1$ is commonly known as the *slope*, and $w_0$ is commonly known as the *intercept*.

Consider the following data, which is scattered about a line with a slope of 2 and an intercept of -5:

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
t = 2 * x - 5 + rng.randn(50)
plt.scatter(x, t);

We can use Scikit-Learn's ``LinearRegression`` estimator to fit this data and construct the best-fit line:

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

model.fit(x[:, np.newaxis], t)

xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

plt.scatter(x, t)
plt.plot(xfit, yfit);
plt.savefig('./img/linear_fit.png')

The slope and intercept of the data are contained in the model's fit parameters, which in Scikit-Learn are always marked by a trailing underscore.
Here the relevant parameters are ``coef_`` and ``intercept_``:

In [ ]:
print("Model slope:    ", model.coef_[0])
print("Model intercept:", model.intercept_)

We see that the results are very close to the inputs, as we might hope.

The ``LinearRegression`` estimator is much more capable than this, however—in addition to simple straight-line fits, it can also handle multidimensional linear models of the form
$$
y = w_0 + w_1 x_1 + w_2 x_2 + \cdots
$$
where there are multiple $x$ values.
Geometrically, this is akin to fitting a plane to points in three dimensions, or fitting a hyper-plane to points in higher dimensions.

The multidimensional nature of such regressions makes them more difficult to visualize, but we can see one of these fits in action by building some example data, using NumPy's matrix multiplication operator:

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
t = 0.5 + np.dot(X, [1.5, -2., 1.])

model.fit(X, t)
print("Model intercept:", model.intercept_)
print("Model slopes:   ", model.coef_)

Here the $t$ (target) data is constructed from three random $x$ values, and the linear regression recovers the coefficients used to construct the data.

In this way, we can use the single ``LinearRegression`` estimator to fit lines, planes, or hyperplanes to our data.
It still appears that this approach would be limited to strictly linear relationships between variables, but it turns out we can relax this as well.

## Basis Function Regression

One trick you can use to adapt linear regression to nonlinear relationships between variables is to transform the data according to *basis functions*.
We will use the ``PolynomialRegression`` pipeline..
The idea is to take our multidimensional linear model:
$$
y = w_0 + w_1 x_1 + w_2 x_2 + w_3 x_3 + \cdots
$$
and build the $x_1, x_2, x_3,$ and so on, from our single-dimensional input $x$.
That is, we let $x_n = f_n(x)$, where $f_n()$ is some function that transforms our data.

For example, if $f_n(x) = x^n$, our model becomes a polynomial regression:
$$
y = w_0 + w_1 x + w_2 x^2 + w_3 x^3 + \cdots
$$
Notice that this is *still a linear model*—the linearity refers to the fact that the coefficients $w_n$ never multiply or divide each other.
What we have effectively done is taken our one-dimensional $x$ values and projected them into a higher dimension, so that a linear fit can fit more complicated relationships between $x$ and $y$.

### Polynomial basis functions

This polynomial projection is useful enough that it is built into Scikit-Learn, using the ``PolynomialFeatures`` transformer:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

We see here that the transformer has converted our one-dimensional array into a three-dimensional array by taking the exponent of each value.
This new, higher-dimensional data representation can then be plugged into a linear regression.

The cleanest way to accomplish this is to use a pipeline.
Let's make a 7th-degree polynomial model in this way:

In [ ]:
from sklearn.pipeline import make_pipeline
poly_model = make_pipeline(PolynomialFeatures(7),
                           LinearRegression())

With this transform in place, we can use the linear model to fit much more complicated relationships between $x$ and $y$. 
For example, here is a sine wave with noise:

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
t = np.sin(x) + 0.1 * rng.randn(50)
poly_model.fit(x[:, np.newaxis], t)

yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, t)
plt.plot(xfit, yfit);

Our linear model, through the use of 7th-order polynomial basis functions, can provide an excellent fit to this non-linear data!

## Regularization

The introduction of basis functions into our linear regression makes the model much more flexible, but it also can very quickly lead to over-fitting (refer back to [Hyperparameters and Model Validation](05.03-Hyperparameters-and-Model-Validation.ipynb) for a discussion of this).
For example, if we choose too many Gaussian basis functions, we end up with results that don't look so good:

In [ ]:
model = make_pipeline(PolynomialFeatures(16),
                      LinearRegression())
model.fit(x[:, np.newaxis], t)
yfit = model.predict(xfit[:, np.newaxis])

plt.scatter(x, t)
plt.plot(xfit, yfit)

plt.xlim(0, 10)
plt.ylim(-1.5, 1.5);

With the data projected to the 16-dimensional basis, the model has far too much flexibility and goes to extreme values between locations where it is constrained by data.
We can see the reason for this if we plot the coefficients of the polynomial bases:

In [ ]:
def basis_plot(model, title=None):
    fig, ax = plt.subplots(1,2, sharex=False, figsize=(20,5))
    model.fit(x[:, np.newaxis], t)
    ax[0].scatter(x, t)
    ax[0].plot(xfit, model.predict(xfit[:, np.newaxis]))
    ax[0].set(xlabel='x', ylabel='y', ylim=(-1.5, 1.5))
    
    if title:
        ax[0].set_title(title)

    ax[1].plot(model.steps[0][1].powers_,
               model.steps[1][1].coef_)
    ax[1].set(xlabel='basis power',
              ylabel='coefficient')
    
model = make_pipeline(PolynomialFeatures(16), LinearRegression())
basis_plot(model)

The left panel of this figure shows the amplitude of the basis function for each power of x.
This is typical over-fitting behavior: the coefficients of higher order basis functions oscillate and cancel each other out.
We know that such behavior is problematic, and it would be nice if we could limit such spikes expliticly in the model by penalizing large values of the model parameters.
Such a penalty is known as *regularization*, and comes in several forms.

### Ridge regression ($L_2$ Regularization)

Perhaps the most common form of regularization is known as *ridge regression* or $L_2$ *regularization*, sometimes also called *Tikhonov regularization*.
This proceeds by penalizing the sum of squares (2-norms) of the model coefficients; in this case, the penalty on the model fit would be 
$$
P = \alpha\sum_{n=1}^N w_n^2
$$
where $\alpha$ is a free parameter that controls the strength of the penalty.
This type of penalized model is built into Scikit-Learn with the ``Ridge`` estimator:

In [ ]:
from sklearn.linear_model import Ridge
model = make_pipeline(PolynomialFeatures(16), Ridge(alpha=0.08))
basis_plot(model, title='Ridge Regression')

The $\alpha$ parameter is essentially a knob controlling the complexity of the resulting model.
In the limit $\alpha \to 0$, we recover the standard linear regression result; in the limit $\alpha \to \infty$, all model responses will be suppressed. Just play with different values of $\alpha$ and see how the resulting fit changes, and what happens to the weights of the coefficients in the lower panel.

One advantage of ridge regression in particular is that it can be computed very efficiently—at hardly more computational cost than the original linear regression model.

### Lasso regression ($L_1$ regularization)

Another very common type of regularization is known as lasso, and involves penalizing the sum of absolute values (1-norms) of regression coefficients:
$$
P = \alpha\sum_{n=1}^N |w_n|
$$
Though this is conceptually very similar to ridge regression, the results can differ surprisingly: for example, due to geometric reasons lasso regression tends to favor *sparse models* where possible: that is, it preferentially sets model coefficients to exactly zero.

We can see this behavior in duplicating the ridge regression figure, but using L1-normalized coefficients:

In [ ]:
from sklearn.linear_model import Lasso
model = make_pipeline(PolynomialFeatures(16), Lasso(alpha=0.001, max_iter=100000))
basis_plot(model, title='Lasso Regression')

With the lasso regression penalty, the majority of the coefficients are exactly zero, with the functional behavior being modeled by a small subset of the available basis functions.
As with ridge regularization, the $\alpha$ parameter tunes the strength of the penalty, and should be determined via, for example, cross-validation.

## Variance and Bias

Let's have a look again at the problem of bias and variance of a Machine Learning model. This is related to the problem of underfitting and overfitting. The more complex your model is, the more variation in the data you can fit. Unfortunately, this also includes noise in the training data, i.e. variations that are not coming from the underlying signal you are interested in. On the other hand, if your model cannot represent the signal, it will fail to fit the general shape of your data (think a linear model fitted to a nonlinear function such as the sine function).

a reminder from Wikipedia: In statistics and machine learning, the bias–variance tradeoff (or dilemma) is the problem of simultaneously minimizing two sources of error that prevent supervised learning algorithms from generalizing beyond their training set:
- The bias is an error from erroneous assumptions in the learning algorithm. High bias can cause an algorithm to miss the relevant relations between features and target outputs (underfitting).
- The variance is an error from sensitivity to small fluctuations in the training set. High variance can cause an algorithm to model the random noise in the training data, rather than the intended outputs (overfitting).

By playing with the number of polynomial basis functions (e.g. $x, x^2, x^3, ...$), also called the degree of the polynomial, you can produce fits of varying complexity to the data. The degree can be changed via the **PolynomialFeatures(n)** option. You can play with this option below and observe the under- or over-fitting behaviour of the result.

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(16),
                           LinearRegression())

rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
t = np.sin(x) + 0.1 * rng.randn(50)

poly_model.fit(x[:, np.newaxis], t)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, t)
plt.plot(xfit, yfit);

To visualize the variance and the bias of the estimate, we can sample a number of datasets and compute how well the model fits and how much the fitted coefficients vary:
- To analyze the bias of the model, we compute the mean squared error between prediction and the target value y.
- To visualize the variance we can look at the variation of the fitted models. In order to compare models of different polynomial degrees, we will only look at the first coefficient for x (i.e. $w_1$) which should exist in all models.

You can experiment with the code below by changing:
- The complexity of the model, i.e. the number of polynomial kernels: ```n_polynomial```
    - ```n_polynomial = 1``` gives us just the linear model
    - ```n_polynomial = 9``` uses coefficients $x, x^2, x^3, ..., x^9$ etc..
- With ```n_samples``` you can change the number of data points used in each experiment.
- With ```n_experiments``` you can set the number of experiments, i.e. how often you want to draw a random dataset and fit the model.

The code below will output two histograms: the left plot shows the variation of $w_1$ across the experiments; the right plot shows how the mean-squared-error is distributed.

Let's start with a linear model (a polynomial of degree 1). What would you expect the bias and variance to look like? 

In [ ]:
n_polynomial = 1
n_samples = 50
n_experiments = 200

poly_model = make_pipeline(PolynomialFeatures(n_polynomial),
                           LinearRegression())

fitted_coeffs = np.zeros((n_experiments,n_polynomial+1))
mse = np.zeros(n_experiments)

for i in range(n_experiments):
    rng = np.random.RandomState()
    x = 10 * rng.rand(n_samples)
    t = np.sin(x) + 0.1 * rng.randn(n_samples)

    poly_model.fit(x[:, np.newaxis], t)
    fitted_coeffs[i] = poly_model.steps[1][1].coef_
    yfit=poly_model.predict(x[:,np.newaxis])
    mse[i]=np.sum((yfit-t)**2)

plt.figure(1)
ax1=plt.subplot(121)
ax1.set_title("coefficient w1")
plt.hist(fitted_coeffs[:,1],30,range=[-2, 2])
ax2=plt.subplot(122)
ax2.set_title("mean squared error of prediction")
plt.hist(mse, 30, range=[0,50])

Now let's use a more complex model (a polynomial of degree 9). Will it have less or more bias (i.e. would it fit each sampled dataset better or worse)? What about the variance?

In [ ]:
n_polynomial = 9
n_samples = 50
n_experiments = 200

poly_model = make_pipeline(PolynomialFeatures(n_polynomial),
                           LinearRegression())

fitted_coeffs = np.zeros((n_experiments,n_polynomial+1))
mse = np.zeros(n_experiments)

for i in range(n_experiments):
    rng = np.random.RandomState()
    x = 10 * rng.rand(n_samples)
    t = np.sin(x) + 0.1 * rng.randn(n_samples)

    poly_model.fit(x[:, np.newaxis], t)
    fitted_coeffs[i] = poly_model.steps[1][1].coef_
    yfit=poly_model.predict(x[:,np.newaxis])
    mse[i]=np.sum((yfit-t)**2)

plt.figure(1)
ax1=plt.subplot(121)
ax1.set_title("coefficient w1")
plt.hist(fitted_coeffs[:,1],30,range=[-5, 5])
ax2=plt.subplot(122)
ax2.set_title("mean squared error of prediction")
plt.hist(mse, 30, range=[0,50])

    

So the simple linear model doesn't fit the data well: it has a high bias. But the coefficients remain stable when you sample new data and fit again (low variance). The complex model accurately fits each sampled dataset (too good actually): it has a low bias. But the estimated coefficients vary a lot between each dataset you sample, so this indicates overfitting: If you have to change your parameters to each new dataset, you probably didn't capture the underlying structure of the data. Those overfitted models with very well approximate your training data but they won't predict new (unseen) data very well. The best way to estimate the expected behavior of models on new data is by using a separate training set (for fitting the model) and a test set (to validate its generalization performance). 

### things to play with...

- Test models of varying complexity and compare the results.
- Increase number of samples used for fitting.
    - How does it influence overfitting for polynomials of higher order?
- Create a separate training and test set.
    - Use the training set to fit the model and the test set to validate the performance.
    - Test polyomial regression models of increasing order. How do the different models generalize (i.e. how well do they perform on the test set)?
    - How does regularization influence the generalization of your models?

## Example: Predicting Bicycle Traffic

As an example, let's take a look at whether we can predict the number of bicycle trips across Seattle's Fremont Bridge based on weather, season, and other factors.

We will join the bike data with another dataset, and try to determine the extent to which weather and seasonal factors—temperature, precipitation, and daylight hours—affect the volume of bicycle traffic through this corridor.
Fortunately, the NOAA makes available their daily [weather station data](http://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND) (I used station ID USW00024233) and we can easily use Pandas to join the two data sources.
We will perform a simple linear regression to relate weather and other information to bicycle counts, in order to estimate how a change in any one of these parameters affects the number of riders on a given day.

In particular, this is an example of how the tools of Scikit-Learn can be used in a statistical modeling framework, in which the parameters of the model are assumed to have interpretable meaning.
As discussed previously, this is not a standard approach within machine learning, but such interpretation is possible for some models.

Let's start by loading the two datasets, indexing by date:

In [ ]:
# !curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

![](img/fremont-bridge.jpg)

In [ ]:
import pandas as pd
daily = pd.read_csv('data/daily_counts_weather.csv', index_col='Date', parse_dates=True)

In [ ]:
daily.head()

With this in place, we can choose the columns to use, and fit a linear regression model to our data.
We will set ``fit_intercept = False``, because the daily flags essentially operate as their own day-specific intercepts:

In [ ]:
# Drop any rows with null values
daily.dropna(axis=0, how='any', inplace=True)

#put in the columns you want to use as predictor variables, for example column_names = ['daylight_hrs','holiday','PRCP'] or column_names=list(daily.columns) for all entries

column_names = ['daylight_hrs']


X = daily[column_names]
y = daily['Total']

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
daily['predicted'] = model.predict(X)

Finally, we can compare the total and predicted bicycle traffic visually:

In [ ]:
daily[['Total', 'predicted']].plot(alpha=0.5);

A simple regression on the hours of daylight at least partially explains the general trend. Now just go back and add more predictor variables and see how the fit changes. 

It is evident that we have missed some key features, especially during the summer time.
Either our features are not complete (i.e., people decide whether to ride to work based on more than just these) or there are some nonlinear relationships that we have failed to take into account (e.g., perhaps people ride less at both high and low temperatures).
Nevertheless, our rough approximation is enough to give us some insights, and we can take a look at the coefficients of the linear model to estimate how much each feature contributes to the daily bicycle count:

In [ ]:
params = pd.Series(model.coef_, index=X.columns)
params

These numbers are difficult to interpret without some measure of their uncertainty.
We can compute these uncertainties quickly using bootstrap resamplings of the data:

In [ ]:
from sklearn.utils import resample
np.random.seed(1)
err = np.std([model.fit(*resample(X, y)).coef_
              for i in range(1000)], 0)

With these errors estimated, let's again look at the results:

In [ ]:
print(pd.DataFrame({'effect': params.round(0),
                    'error': err.round(0)}))

The effect column will show you for each variable how the total number of people taking the bike changes if you change the variable by 1 unit. E.g. for each additional hour of daylight there are 223 more people (on their bikes) crossing the bridge.  

Our model is almost certainly missing some relevant information. For example, nonlinear effects (such as effects of precipitation *and* cold temperature) and nonlinear trends within each variable (such as disinclination to ride at very cold and very hot temperatures) cannot be accounted for in this model.
Additionally, we have thrown away some of the finer-grained information (such as the difference between a rainy morning and a rainy afternoon), and we have ignored correlations between days (such as the possible effect of a rainy Tuesday on Wednesday's numbers, or the effect of an unexpected sunny day after a streak of rainy days).
These are all potentially interesting effects, and you now have the tools to begin exploring them if you wish!

# Additional topics (for those who are interested)

### 01 - Gaussian basis functions

Of course, other basis functions are possible.
For example, one useful pattern is to fit a model that is not a sum of polynomial bases, but a sum of Gaussian bases.
The result might look something like the following figure:

![](./img/05.06-gaussian-basis.png)

The shaded regions in the plot are the scaled basis functions, and when added together they reproduce the smooth curve through the data.
These Gaussian basis functions are not built into Scikit-Learn, but we can write a custom transformer that will create them, as shown here and illustrated in the following figure (Scikit-Learn transformers are implemented as Python classes; reading Scikit-Learn's source is a good way to see how they can be created):

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
    
    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))
        
    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
        
    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_,
                                 self.width_, axis=1)
    
gauss_model = make_pipeline(GaussianFeatures(20),
                            LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)
plt.xlim(0, 10);

We put this example here just to make clear that there is nothing magic about polynomial basis functions: if you have some sort of intuition into the generating process of your data that makes you think one basis or another might be appropriate, you can use them as well.

In [ ]:
model = make_pipeline(GaussianFeatures(30),
                      LinearRegression())
model.fit(x[:, np.newaxis], y)

plt.scatter(x, y)
plt.plot(xfit, model.predict(xfit[:, np.newaxis]))

plt.xlim(0, 10)
plt.ylim(-1.5, 1.5);

In [ ]:
def basis_plot_gauss(model, title=None):
    fig, ax = plt.subplots(1,2,figsize=(20,5))
    model.fit(x[:, np.newaxis], y)
    ax[0].scatter(x, y)
    ax[0].plot(xfit, model.predict(xfit[:, np.newaxis]))
    ax[0].set(xlabel='x', ylabel='y')
    
    if title:
        ax[0].set_title(title)

    ax[1].plot(model.steps[0][1].centers_,
               model.steps[1][1].coef_)
    ax[1].set(xlabel='basis location',
              ylabel='coefficient')
    
model = make_pipeline(GaussianFeatures(30), LinearRegression())
basis_plot_gauss(model)

### things to test

- Use regularization as above in the polynomial examples.
- What happens to the coefficients when you use regularization?

## 02 - Create data frames with pandas

In [ ]:
import pandas as pd
counts = pd.read_csv('data/FremontBridge.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv('data/BicycleWeather.csv', index_col='DATE', parse_dates=True)

Let's get a quick overview of the data. Each pandas dataframe has the function .head() that shows the first couple of rows.

In [ ]:
counts.head()

Next we will compute the total daily bicycle traffic, and put this in its own dataframe:

In [ ]:
daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns

Summing the counts for each day can be done in pandas using _resample_.

In [ ]:
daily=counts.resample('d').sum()
daily.head()

In [ ]:
daily['Total'] = daily.sum(axis=1) #sum counts for both columns (i.e. both sides of the bridge)
daily = daily[['Total']] # remove other columns
daily.head()

The patterns of use generally vary from day to day; let's account for this in our data by adding binary columns that indicate the day of the week:

In [ ]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

In [ ]:
daily.head()

Similarly, we might expect riders to behave differently on holidays; let's add an indicator of this as well:

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
daily['holiday'].fillna(0, inplace=True)

We also might suspect that the hours of daylight would affect how many people ride; let's use the standard astronomical calculation to add this information:

In [ ]:
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))
daily[['daylight_hrs']].plot()
plt.ylim(8, 17)

We can also add the average temperature and total precipitation to the data.
In addition to the inches of precipitation, let's add a flag that indicates whether a day is dry (has zero precipitation):

In [ ]:
# temperatures are in 1/10 deg C; convert to C
weather['TMIN'] /= 10
weather['TMAX'] /= 10

weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# precip is in 1/10 mm; convert to inches
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)

daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])

Finally, let's add a counter that increases from day 1, and measures how many years have passed.
This will let us measure any observed annual increase or decrease in daily crossings:

In [ ]:
daily['annual'] = (daily.index - daily.index[0]).days / 365.

Now our data is in order, and we can take a look at it:

In [ ]:
daily.head()

## plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(0),
                           LinearRegression())

rng = np.random.RandomState(1)
x_train = np.linspace(0,10,10)
y_train = np.sin(x_train) + 0.1 * rng.randn(10)

rng = np.random.RandomState(2)
x_test = np.linspace(1,8,10)
y_test = np.sin(x_test) + 0.1 * rng.randn(10)

xfit = np.linspace(0, 10, 1000)

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-0.png')

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(1),
                           LinearRegression())

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-1.png')

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(4),
                           LinearRegression())

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-4.png')

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(9),
                           LinearRegression())

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-9.png')

In [ ]:
def validate_model(degree):
    poly_model = make_pipeline(PolynomialFeatures(degree),
                           LinearRegression())

    poly_model.fit(x_train[:, np.newaxis], y_train)
    y_fit = poly_model.predict(x_train[:, np.newaxis])
    y_fit_test = poly_model.predict(x_test[:,np.newaxis])
    return np.asarray([np.sum((y_fit-y_train)**2), np.sum((y_fit_test-y_test)**2)])


In [ ]:
res=np.asarray([validate_model(i) for i in range(0,13)])

In [ ]:
res=np.transpose(res)

In [ ]:
plt.figure(figsize=(8,5))

plt.plot(res[0,:])
plt.plot(res[1,:])
plt.legend(('training error','test error'))
plt.savefig('img/polynomial-fit-training-vs-test.png')


In [ ]:
poly_model = make_pipeline(PolynomialFeatures(9),
                           LinearRegression())
n_samples=100

rng = np.random.RandomState(1)
x_train = np.linspace(0,10,n_samples)
y_train = np.sin(x_train) + 0.1 * rng.randn(n_samples)

xfit = np.linspace(0, 10, 1000)

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-9-'+str(n_samples) +'.png')

In [ ]:
from sklearn.linear_model import Ridge
alpha = 0.1

poly_model = make_pipeline(PolynomialFeatures(9),
                          Ridge(alpha=alpha))

n_samples=10

rng = np.random.RandomState(1)

x_train = np.linspace(0,10,n_samples)
y_train = np.sin(x_train) + 0.1 * rng.randn(n_samples)

xfit = np.linspace(0, 10, 1000)

poly_model.fit(x_train[:, np.newaxis], y_train)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(8,5))
plt.scatter(x_train, y_train)
plt.plot(xfit, np.sin(xfit))
plt.plot(xfit, yfit)
plt.savefig('./img/polynomial-fit-sine-order-9-'+str(n_samples) +'-regularized-'+str(alpha) +'.png')